# Data Engineer Take Home Exercise

## Question
### Data Prep

Write a script to transform input CSV to desired output CSV and Parquet. 

You will find a CSV file in the files folder under `data.csv`. There are three steps to this part of the test. Each step concerns manipulating the values for a single field according to the step's requirements. The steps are as follows:

**String cleaning** - The bio field contains text with arbitrary padding, spacing and line breaks. Normalize these values to a space-delimited string.

**Code swap** - There is a supplementary CSV in the files folder under `state_abbreviations`. This "data dictionary" contains state abbreviations alongside state names. For the state field of the input CSV, replace each state abbreviation with its associated state name from the data dictionary.

**Date offset** - The start_date field contains data in a variety of formats. These may include e.g., "June 23, 1912" or "5/11/1930" (month, day, year). But not all values are valid dates. Invalid dates may include e.g., "June 2018", "3/06" (incomplete dates) or even arbitrary natural language. Add a start_date_description field adjacent to the start_date column to filter invalid date values into. Normalize all valid date values in start_date to ISO 8601 (i.e., YYYY-MM-DD).

Your script should take `data.csv` as input and produce a cleansed `enriched.csv` and `enriched.snappy.parquet` files according to the step requirements above.

## Submission Guidelines
We ask that your solutions be implemented in Python (3.8 or newer) or PySpark (3.3 or newer). If you would like to present skills for both approach, feel free to prepare two separate jupyter notebooks. Assume that code will be used monthly to process the data and store it in AWS S3 based data lake. With that assumption please prepare for discussion how this code can be scheduled and how outputs should be stored in S3 bucket.

### Assessment Criteria
Our goal is not to fool you. On the contrary, we would like to see you in your best light! We value clean, DRY and documented code; and in the interest of full disclosure, our assessment criteria is outlined below (in order of significance):

1. Your ability to effectively solve the problems posed.
1. Your ability to solve these problems in a clear and logical manner, with tasteful design.
1. Your ability to appropriately document and comment your code.




# Project submission - Interview challenge (Pandas)

Author: Jakub Pitera

This is a secondary notebook where submission has been converted to Pandas

Files in the main directory:
- **takehomefile.ipynb** - Main Jupyter notebook with assignment details. It showcases how author created the ETL pipeline with Spark according to the data preperation requirements. 
- **ETL.py** - ETL code reformatted into functions. Can be initialized in terminal.
- **ETL_S3.py** - Alternate version of ETL.py. ETL steps were updated to load data to AWS S3 bucket.
- **dl.cfg** - Template for storing AWS credentials used when writing to S3.
- **airflow_dag_naive.py** - using airflow to run ETL tasks. naive approach. demonstrates usage of airflow 
- **airflow_dag_sparky.py** - using airflow to run spark application. more elegant approach. data is loaded to S3
- **takehomefile_pandas.ipynb** - Alternative Jupyter Notebook with ETL converted to Pandas
- **requirements.txt** - List of required packages
- **data.csv** - raw dataset in CSV format.
- **state_abbreviations.csv** - abbreviations data dictionary in CSV format.
- **enriched.csv** - directory storing final enriched ouput in CSV format
- **enriched.snappy.parquet** - directory storing final enriched output in SNAPPY.PARQUET format

## Initialization

In [1]:
# Import libraries
import pandas as pd

## Load dataset

In [2]:
# Read input data to spark dataframe
filepath = 'data.csv'

df = pd.read_csv(filepath)

# Create copy of raw data for reference
df_raw = df.copy(deep=True)

In [3]:
# Inspect if data has been loaded correctly
df.head(10)

,name,gender,birthdate,address,city,state,zipcode,email,bio,job,start_date
0,Leslee Corwin,M,1974-02-01,4933 Weber Walks,Lake Carey,KS,32725,hansen.kennedy@yahoo.com,At aut velit unde minus recusandae molestias. ...,Education administrator,10/06
1,Orris Kuvalis,M,1997-01-08,092 Kanye Forge,South Doshiamouth,TN,8955,nicky.brown@yahoo.com,Corporis non harum doloribus ab provident.\t A...,Industrial buyer,Voluptatem odio.
2,Afton Hirthe,M,1970-08-25,355 Shaquille Centers Suite 834,Lorriborough,OK,12027,noelle.gibson@lebsack.biz,Sed vitae dolorem quae totam sequi fuga odit.\...,Multimedia specialist,Est sed et suscipit.
3,Olinda Wisoky,F,2007-01-11,48328 Rudolph Harbors,Braunport,NE,19620,desirae.ritchie@yahoo.com,Nostrum impedit nulla vero ullam ad repudianda...,Financial controller,06/71
4,Dr. Annmarie Schmitt PhD,M,1995-09-29,47861 Satterfield Meadow Suite 420,Bergstromshire,NC,44200,johnson.betsey@yahoo.com,Commodi quia facere dolores facere. Sed culpa ...,Chartered management accountant,05/02
5,Mathew Grady,F,1973-04-30,59765 Berge Coves Suite 085,Port Jarretbury,SD,94969,legros.kamryn@yahoo.com,Non omnis fugit molestias.\r\n Dolor eum et ev...,"Designer, multimedia",10/95
6,Ailene Abernathy,F,2002-06-19,26158 Tea Crest Suite 735,Ivannachester,MN,41350,diallo72@huels.com,Eius quas expedita ut culpa doloribus.\r\n Et ...,Personal assistant,Ea nostrum et.
7,Justen Carroll,M,2003-08-27,2194 Parker Cove Apt. 737,North Marlo,MD,936,floy.adams@lindgrenmoen.net,Optio molestias accusamus quos aut beatae laud...,"Engineer, mining",10/20/1994
8,Nikita Torphy,M,1979-08-02,65938 Alvira Prairie,Mariyahfort,NH,97295,ocarter@hotmail.com,Recusandae quod sed provident consequatur. Ad ...,Photographer,09/74
9,Murry Waelchi PhD,F,2013-12-03,4431 Sheridan Divide,Port Markellview,CT,14333,georgia.rice@hotmail.com,Perspiciatis aut autem ea et odit.\t Quo nulla...,Software engineer,Quibusdam similique.


In [4]:
df[['name', 'gender', 'birthdate']].head(100)

,name,gender,birthdate
0,Leslee Corwin,M,1974-02-01
1,Orris Kuvalis,M,1997-01-08
2,Afton Hirthe,M,1970-08-25
3,Olinda Wisoky,F,2007-01-11
4,Dr. Annmarie Schmitt PhD,M,1995-09-29
...,...,...,...
95,Octavie DuBuque,M,1989-12-27
96,Augustina Gibson PhD,M,1973-05-07
97,Dr. Eino Schaefer I,F,1980-12-29
98,Dwayne Lynch,M,2013-01-10


Pandas managed to parse all columns accurately.

## Inspecting dataset

In [5]:
# Schema
df.dtypes

name          object
gender        object
birthdate     object
address       object
city          object
state         object
zipcode        int64
email         object
bio           object
job           object
start_date    object
dtype: object

In [6]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Dtype'])

,Dtype
name,object
gender,object
birthdate,object
address,object
city,object
state,object
zipcode,int64
email,object
bio,object
job,object


In [7]:
# Column statistics
df.describe()

,zipcode
count,500.000000
mean,48102.664000
std,28926.117961
min,601.000000
25%,22455.000000
50%,46668.500000
75%,72885.000000
max,99877.000000


In [8]:
# Dataframe shape
rows, cols = df.shape
print("Dataframe row count: ", rows)
print("Dataframe columns count: ", cols)

Dataframe row count:  500
Dataframe columns count:  11


In [9]:
# Number of missing and values in df for each column
df.isna().sum()

name          0
gender        0
birthdate     0
address       0
city          0
state         0
zipcode       0
email         0
bio           0
job           0
start_date    0
dtype: int64

## Transformation

### String cleaning
The bio field contains text with arbitrary padding, spacing and line breaks. Normalize these values to a space-delimited string.

In [10]:
# Inspect 'bio' column
df['bio'].head(100)

0     At aut velit unde minus recusandae molestias. ...
1     Corporis non harum doloribus ab provident.\t A...
2     Sed vitae dolorem quae totam sequi fuga odit.\...
3     Nostrum impedit nulla vero ullam ad repudianda...
4     Commodi quia facere dolores facere. Sed culpa ...
                            ...                        
95    Libero minima dolor sit quos occaecati.\r\n De...
96    Repellendus cupiditate totam non officia.\r\n\...
97    Eaque dignissimos iusto praesentium. Velit vol...
98    Debitis enim autem beatae unde ad rerum saepe ...
99    Eius id laborum sint.\t           Dolorem enim...
Name: bio, Length: 100, dtype: object

In [11]:
# Print whole lines 
n = 10

for i in range(0,n):
    print(i, df['bio'][i])

0 At aut velit unde minus recusandae molestias. Est maxime labore nostrum.	 Vero debitis neque doloremque accusantium incidunt corporis et et.
1 Corporis non harum doloribus ab provident.	 Alias autem error id modi saepe. Ut delectus fugit dolores.
     
2 Sed vitae dolorem quae totam sequi fuga odit.	        Eaque alias quisquam blanditiis veniam. Aut perferendis sint deleniti accusamus quod.	
3 Nostrum impedit nulla vero ullam ad repudiandae. Excepturi praesentium tempore aspernatur ea est.	 Ipsa alias molestiae rerum omnis voluptates ut.	    
4 Commodi quia facere dolores facere. Sed culpa sit quo.	 Exercitationem error aut odio possimus.
5 Non omnis fugit molestias.
 Dolor eum et eveniet soluta eum. Placeat sapiente temporibus perspiciatis tempora quae quia.		
6 Eius quas expedita ut culpa doloribus.
 Et laboriosam quidem repellendus eveniet a. Nostrum soluta corporis doloremque sint est excepturi quisquam.	

7 Optio molestias accusamus quos aut beatae laudantium qui.	
 Veniam reru

In [12]:
# Normalize escape characters, padding and spacing with a single space
df['bio'] = (df['bio'].str.replace('\\[tbnrfsu]', ' ', regex=True)
                      .str.replace('\s+', ' ', regex=True)
                      .str.strip())

In [13]:
# Confirm the changes were applied
df['bio'].head(100)

0     At aut velit unde minus recusandae molestias. ...
1     Corporis non harum doloribus ab provident. Ali...
2     Sed vitae dolorem quae totam sequi fuga odit. ...
3     Nostrum impedit nulla vero ullam ad repudianda...
4     Commodi quia facere dolores facere. Sed culpa ...
                            ...                        
95    Libero minima dolor sit quos occaecati. Delect...
96    Repellendus cupiditate totam non officia. Mole...
97    Eaque dignissimos iusto praesentium. Velit vol...
98    Debitis enim autem beatae unde ad rerum saepe ...
99    Eius id laborum sint. Dolorem enim ut non volu...
Name: bio, Length: 100, dtype: object

In [14]:
# Print whole lines 
n = 10

for i in range(0,n):
    print(i, df['bio'][i])

0 At aut velit unde minus recusandae molestias. Est maxime labore nostrum. Vero debitis neque doloremque accusantium incidunt corporis et et.
1 Corporis non harum doloribus ab provident. Alias autem error id modi saepe. Ut delectus fugit dolores.
2 Sed vitae dolorem quae totam sequi fuga odit. Eaque alias quisquam blanditiis veniam. Aut perferendis sint deleniti accusamus quod.
3 Nostrum impedit nulla vero ullam ad repudiandae. Excepturi praesentium tempore aspernatur ea est. Ipsa alias molestiae rerum omnis voluptates ut.
4 Commodi quia facere dolores facere. Sed culpa sit quo. Exercitationem error aut odio possimus.
5 Non omnis fugit molestias. Dolor eum et eveniet soluta eum. Placeat sapiente temporibus perspiciatis tempora quae quia.
6 Eius quas expedita ut culpa doloribus. Et laboriosam quidem repellendus eveniet a. Nostrum soluta corporis doloremque sint est excepturi quisquam.
7 Optio molestias accusamus quos aut beatae laudantium qui. Veniam rerum voluptatibus beatae et facere 

### Code swap 
There is a supplementary CSV in the files folder under `state_abbreviations`. This "data dictionary" contains state abbreviations alongside state names. For the state field of the input CSV, replace each state abbreviation with its associated state name from the data dictionary.

In [15]:
# Option 1 - Creating state abbrevations dict using pandas
abbr_dict = pd.read_csv('state_abbreviations.csv', index_col='state_abbr').to_dict()['state_name']

# Replace 'state' col using dictionary (uncomment line to run this option on dataframe)
df['state'] = df['state'].map(abbr_dict)

In [16]:
# Option 2 - Creating state abbrevations dict using csv
import csv

with open('state_abbreviations.csv', 'r') as f:
    next(f) # skip header line
    abbr_dict = {key: val for key,val in csv.reader(f)}

# Replace 'state' col using dictionary (uncomment line to run this option on dataframe)
# df['state'] = df['state'].map(abbr_dict)

In [17]:
# Confirming results by inspecting 'state' col
df['state'].head(20)

0             Kansas
1          Tennessee
2           Oklahoma
3           Nebraska
4     North Carolina
5       South Dakota
6          Minnesota
7           Maryland
8      New Hampshire
9        Connecticut
10           Arizona
11           Vermont
12          Missouri
13              Iowa
14      North Dakota
15             Texas
16     West Virginia
17          Illinois
18         Tennessee
19       Mississippi
Name: state, dtype: object

### Date offset
The start_date field contains data in a variety of formats. These may include e.g., "June 23, 1912" or "5/11/1930" (month, day, year). But not all values are valid dates. Invalid dates may include e.g., "June 2018", "3/06" (incomplete dates) or even arbitrary natural language. Add a start_date_description field adjacent to the start_date column to filter invalid date values into. Normalize all valid date values in start_date to ISO 8601 (i.e., YYYY-MM-DD).

In [18]:
# Inspect 'start_date' col
df['start_date'].head(50)

0                    10/06
1         Voluptatem odio.
2     Est sed et suscipit.
3                    06/71
4                    05/02
5                    10/95
6           Ea nostrum et.
7               10/20/1994
8                    09/74
9     Quibusdam similique.
10              09/21/1977
11           December 1999
12           July 11, 1995
13     Consequuntur rerum.
14            October 1973
15          September 1998
16     Repellat accusamus.
17          March 12, 2000
18              1999-09-15
19              1970-02-19
20      September 18, 1988
21           Eum adipisci.
22                   02/07
23               July 1994
24     Voluptates tempore.
25                   03/80
26              1991-07-22
27                May 1988
28                   10/02
29                   11/97
30              March 2005
31       Et tempore rerum.
32    Accusamus quia odit.
33      Voluptatibus iste.
34              April 2015
35            Fugiat quia.
36                   04/01
3

Spotted 3 date formats that are valid.
 - "MMMM/DD/YYYY"
 - "YYYY-MM-DD"
 - "MMMM DD, YYYY"

In [19]:
# Create 'sd' col by running to_datetime() with each format on 'start_date' col
df['sd'] = pd.to_datetime(df['start_date'], format="%m/%d/%Y", errors='coerce') \
            .fillna(pd.to_datetime(df['start_date'], format="%Y-%m-%d", errors='coerce')) \
            .fillna(pd.to_datetime(df['start_date'], format="%B %d, %Y", errors='coerce'))

In [20]:
# Inspect 'sd' col
df['sd'].head(20)

0           NaT
1           NaT
2           NaT
3           NaT
4           NaT
5           NaT
6           NaT
7    1994-10-20
8           NaT
9           NaT
10   1977-09-21
11          NaT
12   1995-07-11
13          NaT
14          NaT
15          NaT
16          NaT
17   2000-03-12
18   1999-09-15
19   1970-02-19
Name: sd, dtype: datetime64[ns]

In [21]:
# Check number of not NaT records
df['sd'].notna().sum()

162

In [22]:
# Use 'sd' col to create 'start_date_description' col describing if 'start_date' vals are Valid or Invalid
df['start_date_description'] = 'Valid'
df.loc[df['sd'].isna(), 'start_date_description'] = 'Invalid'

In [23]:
# Verify the number of Valid records in 'start_date_description'
df['start_date_description'].value_counts()

start_date_description
Invalid    338
Valid      162
Name: count, dtype: int64

In [24]:
# As per requirements - the 'start_date' column valid values get normalized while invalid remain unchanged. 
# If it is not what the reviewer had in mind then in 'sd' column invalid values were turned to null 
df['start_date'] = df['sd'].dt.date.fillna(df['start_date'])

In [25]:
# Confirming results by inspecting dataframe
df[['start_date', 'start_date_description', 'sd']].head(40)

,start_date,start_date_description,sd
0,10/06,Invalid,NaT
1,Voluptatem odio.,Invalid,NaT
2,Est sed et suscipit.,Invalid,NaT
3,06/71,Invalid,NaT
4,05/02,Invalid,NaT
5,10/95,Invalid,NaT
6,Ea nostrum et.,Invalid,NaT
7,1994-10-20,Valid,1994-10-20
8,09/74,Invalid,NaT
9,Quibusdam similique.,Invalid,NaT


## QC check
Direct checks on processed columns to confirm successful data preparation. Serving the role of sanity check.

### 'bio' column normalization

In [26]:
# Tests if 'bio' column does not contain \n', '\t', '  ', '   '

# Number of records in bio col with undesired characters
bio_mask = df['bio'].str.contains('\n|\t|  |   ')
bio_count = df['bio'].where(bio_mask).count()

# Check results
if bio_count == 0:
    bio_check = True
    print('QC: bio column normalization - Passed')
else:
    bio_check = False
    print('QC: bio column normalization - FAILED')

QC: bio column normalization - Passed


### 'state' column abbreviation swap

In [27]:
# Tests if all 'state' column strings are larger than 2 

# Number of records in 'state' col with strings length < 3
state_mask = df['state'].str.len() < 3
state_count = df['state'].where(state_mask).count()
                 
# Check results
if state_count == 0:
    state_check = True
    print('QC: state column abbreviation swap - Passed')
else:
    state_check = False
    print('QC: state column abbreviation swap - FAILED')

QC: state column abbreviation swap - Passed


### date offset
Disclaimer:  Hard to verify numerically. All depends whether user defined all valid formats found in raw column.

In [28]:
# Tests if 'sd' staging column dtype is datetime64[ns] 
date_check1 = str(df['sd'].dtypes) == "datetime64[ns]"

# Check results
if date_check1:
    print('QC: date offset - "sd" column is DateType - Passed')
else:
    print('QC: date offset - "sd" column is DateType - FAILED')

QC: date offset - "sd" column is DateType - Passed


In [29]:
# Tests if all valid dates found were converted to ISO 8601 'YYYY-MM-DD' standard

# Number of records with initial date format described as valid 
valid_count = (df['start_date_description'] == 'Valid').sum()
# Number of records in ISO 8601 format in 'start_date' col
isodate_count = pd.to_datetime(df['start_date'], format='ISO8601', errors='coerce').notna().sum()

# Check results
date_check2 = valid_count == isodate_count
if date_check2:
    print('QC: date offset - All Valid dates converted to ISO 8601 - Passed')
else:
    print('QC: date offset - All Valid dates converted to ISO 8601 - FAILED')

QC: date offset - All Valid dates converted to ISO 8601 - Passed


In [30]:
# additional visual check
df[['start_date', 'start_date_description','sd']].sample(50)

,start_date,start_date_description,sd
91,02/02,Invalid,NaT
96,Sed aut suscipit.,Invalid,NaT
27,May 1988,Invalid,NaT
194,2004-01-01,Valid,2004-01-01
14,October 1973,Invalid,NaT
117,Est magnam.,Invalid,NaT
46,July 1991,Invalid,NaT
264,Quia in vero.,Invalid,NaT
62,04/72,Invalid,NaT
278,July 2006,Invalid,NaT


### QC results

In [31]:
# Print SUCCESS if all checks passed
if all((bio_check, state_check, date_check1, date_check2)):
    print("QC Results: SUCCESS")
else:
    print("QC Results: FAILURE")

QC Results: SUCCESS


In [34]:
df.head()

,name,gender,birthdate,address,city,state,zipcode,email,bio,job,start_date,sd,start_date_description
0,Leslee Corwin,M,1974-02-01,4933 Weber Walks,Lake Carey,Kansas,32725,hansen.kennedy@yahoo.com,At aut velit unde minus recusandae molestias. ...,Education administrator,10/06,NaT,Invalid
1,Orris Kuvalis,M,1997-01-08,092 Kanye Forge,South Doshiamouth,Tennessee,8955,nicky.brown@yahoo.com,Corporis non harum doloribus ab provident. Ali...,Industrial buyer,Voluptatem odio.,NaT,Invalid
2,Afton Hirthe,M,1970-08-25,355 Shaquille Centers Suite 834,Lorriborough,Oklahoma,12027,noelle.gibson@lebsack.biz,Sed vitae dolorem quae totam sequi fuga odit. ...,Multimedia specialist,Est sed et suscipit.,NaT,Invalid
3,Olinda Wisoky,F,2007-01-11,48328 Rudolph Harbors,Braunport,Nebraska,19620,desirae.ritchie@yahoo.com,Nostrum impedit nulla vero ullam ad repudianda...,Financial controller,06/71,NaT,Invalid
4,Dr. Annmarie Schmitt PhD,M,1995-09-29,47861 Satterfield Meadow Suite 420,Bergstromshire,North Carolina,44200,johnson.betsey@yahoo.com,Commodi quia facere dolores facere. Sed culpa ...,Chartered management accountant,05/02,NaT,Invalid


In [36]:
df['zipcode'] = df['zipcode'].astype(str)

In [37]:
df.dtypes

name                              object
gender                            object
birthdate                         object
address                           object
city                              object
state                             object
zipcode                           object
email                             object
bio                               object
job                               object
start_date                        object
sd                        datetime64[ns]
start_date_description            object
dtype: object

In [ ]:
split(' ') < 2

In [ ]:
df['zipcode'] = df

In [ ]:
f"{[:2]}-{2:}"

In [38]:
s = '32725'

In [39]:
spol = f"{s[:2]}-{s[2:]}"

In [40]:
spol

'32-725'

In [43]:
df['zipcode_pl'] = df['zipcode'].str[:2] + '-' + df['zipcode'].str[2:]

In [44]:
df['zipcode_pl']

0      32-725
1       89-55
2      12-027
3      19-620
4      44-200
        ...  
495    11-320
496    39-216
497    50-680
498    59-506
499     49-38
Name: zipcode_pl, Length: 500, dtype: object

## Save 

In [32]:
# Write enriched dataframe to csv and snappy.parquet
output_name_csv = 'enriched.csv'
output_name_parquet = 'enriched.snappy.parquet'

# uncomment to actually save
#df.to_csv(output_name_csv, index=False)
#df.to_parquet(output_name_parquet, compression="snappy")